In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
import glob
import os
import os.path
import h5py
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
import random
from torch.optim.lr_scheduler import MultiStepLR
from torch.autograd import Variable

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install wandb --upgrade
import wandb
wandb.login()
#wandb.init(project="Deraining",name = "SSIM_100L_PRN",resume = True,id="21n7u9u9")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 68.0 MB/s 
     |████████████████████████████████| 162 kB 77.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 162 kB 75.7 MB/s 
     |████████████████████████████████| 158 kB 76.5 MB/s 
     |████████████████████████████████| 157 kB 55.2 MB/s 
     |████████████████████████████████| 157 kB 55.5 MB/s 
     |████████████████████████████████| 157 kB 80.8 MB/s 
     |████████████████████████████████| 157 kB 82.4 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 78.4 MB/s 
     |████████████████████████████████| 157 kB 78.3 MB/s 
     |████████████████████████████████| 156 kB 79.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=b3

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="Deraining",name = "SSIM_100L_PRN",resume = True)

wandb: Currently logged in as: messiah2305. Use `wandb login --relogin` to force relogin


In [5]:
def normalize(data):
    return data / 255.

In [6]:
def Im2Patch(img, win, stride=1):
    k = 0
    endc = img.shape[0]
    endw = img.shape[1]
    endh = img.shape[2]
    patch = img[:, 0:endw - win + 0 + 1:stride, 0:endh - win + 0 + 1:stride]
    TotalPatNum = patch.shape[1] * patch.shape[2]
    Y = np.zeros([endc, win * win, TotalPatNum], np.float32)

    for i in range(win):
        for j in range(win):
            patch = img[:, i:endw - win + i + 1:stride, j:endh - win + j + 1:stride]
            Y[:, k, :] = np.array(patch[:]).reshape(endc, TotalPatNum)
            k = k + 1
    return Y.reshape([endc, win, win, TotalPatNum])

In [7]:
patch_size=100
stride=80

list_x = []
list_y = []
patch_size=100
stride=80

input_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_train_rainy")
target_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_train_derained")


train_num = 0

for i in range(200):
    target_file = "%d.jpg" % (i)
    target = cv2.imread(os.path.join(target_path,target_file))
    b, g, r = cv2.split(target)
    target = cv2.merge([r, g, b])
    input_file = "%d.jpg" % (i)
    input = cv2.imread(os.path.join(input_path,input_file))
    b, g, r = cv2.split(input)
    input = cv2.merge([r, g, b])

    #plt.imshow(input)
    #plt.imshow(target)

    target = np.float32(normalize(target))
    target_patches = Im2Patch(target.transpose(2,0,1), win=patch_size, stride=stride)
    for n in range(target_patches.shape[3]):
        target1 = target_patches[:,:,:,n]
        list_x.append(target1)
    #plt.imshow(target1.transpose(1,2,0))
    #plt.imshow(target_patches[:,:,:,0].transpose(1,2,0))

    input = np.float32(normalize(input))
    input_patches = Im2Patch(input.transpose(2, 0, 1), win=patch_size, stride=stride)
    for n in range(input_patches.shape[3]):
        input1 = input_patches[:,:,:,n]
        list_y.append(input1)

    #print("target file: %s # samples: %d" % (input_file, target_patches.shape[3]))
#target_h5f.close()
#input_h5f.close()

In [8]:
%cd /content/drive/My Drive/
np.save("train_100L_Rain.npy",list_y)
np.save("train_100L_Derain.npy",list_x)

/content/drive/My Drive


In [9]:
image_x = np.load("/content/drive/MyDrive/train_100L_Rain.npy")
image_y = np.load("/content/drive/MyDrive/train_100L_Derain.npy")

In [7]:
trans = transforms.Compose([transforms.ToTensor()])
class Custom_Dataset(Dataset):
  def __init__(self,X,Y):
    self.images=X
    self.labels=Y

  def __len__(self):
    return len(self.images)

  def __getitem__(self,i):
    x = self.images[i]
    y = self.labels[i]
    x = trans(x)
   # x = x.permute(1,2,0)
    y = trans(y)
   # y = y.permute(1,2,0)
    return x, y

In [8]:
class PRN(nn.Module):
    def __init__(self, recurrent_iter=6, use_GPU=True):
        super(PRN, self).__init__()
        self.iteration = recurrent_iter

        self.conv0 = nn.Sequential(
            nn.Conv2d(6, 32, 3, 1, 1),
            nn.ReLU()
        )
        self.res_conv1 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU()
        )
        self.res_conv2 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU()
        )
        self.res_conv3 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU()
        )
        self.res_conv4 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU()
        )
        self.res_conv5 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.ReLU()
        )
        self.conv = nn.Sequential(
            nn.Conv2d(32, 3, 3, 1, 1),
        )
    def forward(self, input):

        x = input

        x_list = []
        for i in range(self.iteration):
            x = torch.cat((input, x), 1)
            x = self.conv0(x)
            resx = x
            x = F.relu(self.res_conv1(x) + resx)
            resx = x
            x = F.relu(self.res_conv2(x) + resx)
            resx = x
            x = F.relu(self.res_conv3(x) + resx)
            resx = x
            x = F.relu(self.res_conv4(x) + resx)
            resx = x
            x = F.relu(self.res_conv5(x) + resx)
            x = self.conv(x)

            x = x + input
            x_list.append(x)

        return x, x_list

In [9]:
model = PRN(recurrent_iter=6)

In [10]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [11]:
device = get_default_device()
device
model = model.to(device)

In [12]:
def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += compare_psnr(Iclean[i,:,:,:], Img[i,:,:,:], data_range=data_range)
    return (PSNR/Img.shape[0])

In [13]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from math import exp
 
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    mu1 = F.conv2d(img1, window, padding = window_size//2, groups = channel)
    mu2 = F.conv2d(img2, window, padding = window_size//2, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding = window_size//2, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = window_size//2, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = window_size//2, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)
class SSIM(torch.nn.Module):
    def __init__(self, window_size = 11, size_average = True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.data.type() == img1.data.type():
            window = self.window
        else:
            window = create_window(self.window_size, channel)
            
            if img1.is_cuda:
                window = window.cuda(img1.get_device())
            window = window.type_as(img1)
            
            self.window = window
            self.channel = channel


        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)

def ssim(img1, img2, window_size = 11, size_average = True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    
    return _ssim(img1, img2, window, window_size, channel, size_average)        

In [14]:
def fit(epochs,lr,loader_train):

    criterion = SSIM()
    #criterion = nn.MSELoss(size_average=False)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    #scheduler = MultiStepLR(optimizer, milestones=milestone, gamma=0.2)



    for epoch in range(epochs):
        cum_loss = 0
        cum_psnr = 0
        print("*******************************************************")
        for i, (input_train, target_train) in enumerate(loader_train, 0):
            model.zero_grad()
            optimizer.zero_grad()

            input_train, target_train = Variable(input_train), Variable(target_train)

            input_train, target_train = input_train.to(device), target_train.to(device)

            out_train, _ = model(input_train)
            pixel_metric = criterion(target_train, out_train)
            loss = -pixel_metric
            cum_loss += loss.item()

            loss.backward()
            optimizer.step()
   
            out_train = torch.clamp(out_train, 0., 1.)
            psnr_train = batch_PSNR(out_train, target_train, 1.)

            cum_psnr = cum_psnr + psnr_train

            #print("Loss after iteration ",i," :",loss.item())
            #print("PSNR after iteration ",i," :",psnr_train)

            wandb.log({'Loss_Train_batch':loss,'PSNR_batch':psnr_train})
        print("*******************************************************")
        print("Loss after epoch ",epoch," :",cum_loss/len(loader_train))
        print("PSNR after epoch ",epoch," :",cum_psnr/len(loader_train))
        print("*******************************************************")
        print("*******************************************************")
        wandb.log({'Loss_Train':cum_loss/len(loader_train),'PSNR':cum_psnr/len(loader_train)})
        torch.save(model.state_dict(), "/content/drive/MyDrive/derain100L_pnr_ssimloss_try1.pth")

In [15]:
lr = 0.0001
epochs = 100
#dataset_train = Custom_Dataset(X = image_x,Y=image_y)
#loader_train = DataLoader(dataset=dataset_train,num_workers=2, batch_size=18,shuffle=True)

In [23]:
list_x = []
list_y = []

input_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_train_rainy")
target_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_train_derained")



for i in range(200):
    if (i<9):
      target_file = "%d.jpg" % (i)
      input = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      target = cv2.imread(os.path.join(input_path,input_file)) 

    elif i<99:
      target_file = "%d.jpg" % (i)
      input = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      target = cv2.imread(os.path.join(input_path,input_file)) 

    else :
      target_file = "%d.jpg" % (i)
      input = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      target = cv2.imread(os.path.join(input_path,input_file)) 


    
    b, g, r = cv2.split(target)
    target = cv2.merge([r, g, b])

    b, g, r = cv2.split(input)
    input = cv2.merge([r, g, b]) 
      
    target = np.float32(normalize(target))
    input  = np.float32(normalize(input))
    

    list_x.append(input)
    list_y.append(target)

In [32]:
dataset_train = Custom_Dataset(X = list_y,Y=list_x)
loader_train = DataLoader(dataset=dataset_train,num_workers=2, batch_size=1)

In [16]:
list_x = []
list_y = []

input_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_test_rainy")
target_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_test_derained")



for i in range(100):
    if (i<9):
      target_file = "%d.jpg" % (i)
      input = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      target = cv2.imread(os.path.join(input_path,input_file)) 

    elif i<99:
      target_file = "%d.jpg" % (i)
      input = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      target = cv2.imread(os.path.join(input_path,input_file)) 

    else :
      target_file = "%d.jpg" % (i)
      input = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      target = cv2.imread(os.path.join(input_path,input_file)) 


    
    b, g, r = cv2.split(target)
    target = cv2.merge([r, g, b])

    b, g, r = cv2.split(input)
    input = cv2.merge([r, g, b]) 
      
    target = np.float32(normalize(target))
    input  = np.float32(normalize(input))
    

    list_x.append(input)
    list_y.append(target)

In [17]:
np.save("test_100L_Rain.npy",list_y)
np.save("test_100L_Derain.npy",list_x)

In [18]:
dataset_test = Custom_Dataset(X = list_y,Y=list_x)
loader_test = DataLoader(dataset=dataset_test,num_workers=2, batch_size=1)

In [19]:
cum_loss  = 0
cum_psnr  = 0
criterion = SSIM()
for (input_test, target_test) in loader_test:
            input_test, target_test = Variable(input_test), Variable(target_test)

            input_test, target_test = input_test.to(device), target_test.to(device)

            out_test, _ = model(input_test)

            out_test = torch.clamp(out_test, 0., 1.)
            psnr_test = batch_PSNR(out_test, target_test, 1)
            pixel_metric = criterion(target_test, out_test)
            
            cum_loss += pixel_metric.item()
            cum_psnr = cum_psnr + psnr_test
print("The ssim of the testbset is: ",cum_loss/len(loader_test))
print("The psnr of the testbset is: ",cum_psnr/len(loader_test))
wandb.log({'SSIM_TEST':cum_loss/len(loader_test),'PSNR_TEST':cum_psnr/len(loader_test)})

The ssim of the testbset is:  0.79009250998497
The psnr of the testbset is:  20.9313273290098


In [35]:
fit(epochs=300,lr=lr,loader_train = loader_train)

*******************************************************
*******************************************************
Loss after epoch  0  : -0.7424032226204872
PSNR after epoch  0  : 23.63692115315393
*******************************************************
*******************************************************
*******************************************************
*******************************************************
Loss after epoch  1  : -0.7519756816327572
PSNR after epoch  1  : 24.081301172332395
*******************************************************
*******************************************************
*******************************************************
*******************************************************
Loss after epoch  2  : -0.7599463373422622
PSNR after epoch  2  : 24.385811013372667
*******************************************************
*******************************************************
*******************************************************
******************

KeyboardInterrupt: ignored

In [36]:
for i in range(10):

    input_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_test_rainy")
    target_path = os.path.join("/content/drive/MyDrive/Rain_dataset/Final_test_derained")
    if (i<9):
      target_file = "%d.jpg" % (i)
      inp = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      y = cv2.imread(os.path.join(input_path,input_file))   
    
    else:
      target_file = "%d.jpg" % (i)
      inp = cv2.imread(os.path.join(target_path,target_file))

      input_file = "%d.jpg" % (i)
      y = cv2.imread(os.path.join(input_path,input_file))     
    
    true_y = y
    b, g, r = cv2.split(y)
    y = cv2.merge([r, g, b])
    y = normalize(np.float32(y))
    y = np.expand_dims(y.transpose(2, 0, 1), 0)
    y = torch.Tensor(y)
    y = y.to(device)
    out, _ = model(y)
    save_out = np.uint8(255 * out.data.cpu().numpy().squeeze())  
    save_out = save_out.transpose(1, 2, 0)
    b, g, r = cv2.split(save_out)
    save_out = cv2.merge([r, g, b])
    

    wandb.log({
           "rained_test": wandb.Image(true_y),
           "true_derained_test": wandb.Image(inp),
           "predicted_derained_test": wandb.Image(save_out)
    })

In [30]:
print(np.array(list_x).shape)

(200, 512, 512, 3)
